NODELTA WALA HAI YE

In [1]:
import tensorflow as tf
import os
import numpy as np
import pickle
from tensorflow.keras.layers import Activation
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
### MODEL ###

In [3]:
def activation_fn(x):
    value = (5*(tf.math.exp(x/5) - tf.math.exp(-x/5))/(tf.math.exp(x/5) + tf.math.exp(-x/5)))
    return value

In [4]:
from keras.utils.generic_utils import get_custom_objects
get_custom_objects().update({'modified_tanh': Activation(activation_fn)})

In [5]:
input_layer = tf.keras.layers.Input((32, 5))
lstm_1 = tf.keras.layers.LSTM(16, activation='modified_tanh', recurrent_activation='sigmoid', dropout = 0.15, return_sequences = True, return_state = True)(input_layer)
lstm_2 = tf.keras.layers.LSTM(64, activation='modified_tanh', recurrent_activation='sigmoid', dropout = 0.15, return_sequences = True, return_state = True)(lstm_1[0])
lstm_3 = tf.keras.layers.LSTM(256, activation='modified_tanh', recurrent_activation='sigmoid', dropout = 0.15, return_sequences = True, return_state = True)(lstm_2[0])
lstm_4 = tf.keras.layers.LSTM(512, activation='modified_tanh', recurrent_activation='sigmoid', dropout = 0.15, return_sequences = True, return_state = True)(lstm_3[0])
lstm_5 = tf.keras.layers.LSTM(256, activation='modified_tanh', recurrent_activation='sigmoid', dropout = 0.15, return_sequences = True, return_state = True)(lstm_4[0])
lstm_6 = tf.keras.layers.LSTM(64, activation='modified_tanh', recurrent_activation='sigmoid', dropout = 0.15, return_sequences = True, return_state = True)(lstm_5[0])
lstm_7 = tf.keras.layers.LSTM(16, activation='modified_tanh', recurrent_activation='sigmoid', dropout = 0.15, return_sequences = True, return_state = True)(lstm_6[0])
lstm_8 = tf.keras.layers.LSTM(2, activation='modified_tanh', recurrent_activation='sigmoid', dropout = 0.15, return_sequences = True, return_state = True)(lstm_7[0])
print(lstm_8[0])

KerasTensor(type_spec=TensorSpec(shape=(None, 32, 2), dtype=tf.float32, name=None), name='lstm_7/transpose_1:0', description="created by layer 'lstm_7'")


In [6]:
model = tf.keras.models.Model(inputs = input_layer, outputs = lstm_8[0], name = "DLVS_Model")

In [7]:
model.summary()

Model: "DLVS_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 5)]           0         
_________________________________________________________________
lstm (LSTM)                  [(None, 32, 16), (None, 1 1408      
_________________________________________________________________
lstm_1 (LSTM)                [(None, 32, 64), (None, 6 20736     
_________________________________________________________________
lstm_2 (LSTM)                [(None, 32, 256), (None,  328704    
_________________________________________________________________
lstm_3 (LSTM)                [(None, 32, 512), (None,  1574912   
_________________________________________________________________
lstm_4 (LSTM)                [(None, 32, 256), (None,  787456    
_________________________________________________________________
lstm_5 (LSTM)                [(None, 32, 64), (None, 6 8

In [8]:
### LOSS ###

In [9]:
def loss(y_true, y_pred):
    y_true = tf.convert_to_tensor(y_true, dtype = tf.float32)
    y_pred = tf.convert_to_tensor(y_pred, dtype = tf.float32)
    l1 = tf.reduce_mean(tf.math.abs((y_true - y_pred)))
    l2 = tf.reduce_mean(tf.math.pow((y_true, y_pred), 2))
    loss = l1 + l2
    return loss

In [1]:
import os
error_final = []
vel_final = []
path = "/home/uas-dtu/Documents/DL_IBVS/data_2/"
for a in os.listdir(path):
    file = open(path + "/" + a, "r")
    for i in [file.readlines()]:
        for j in range(len(i) - 32):
#             print(j)
            error_data = []
            vel_data = []
            for k in range(32):
                #print(i[j + k].split(",")[3].split('[')[1].split("]")[0])
                #rint(float(i[j + k].split(",")[5]))
                #print(i[j + k].split(",")[4].split("\n"))
                error_data.append([float(i[j + k].split(",")[0]), 
                                   float(i[j + k].split(",")[1]),
                                   float(i[j + k].split(",")[2]), 
                                   float(i[j + k].split(",")[3]), 
                                   float(i[j + k].split(",")[4])])
                vel_data.append([float(i[j + k].split(",")[5].split('[')[1].split("]")[0]),
                                 float(i[j + k].split(",")[6].split('[')[1].split("]")[0])])
                
                #print(error_data)
            error_final.append(error_data)
            vel_final.append(vel_data)

In [2]:
print(len(vel_final))

177973


In [12]:
import numpy as np
import cv2
from tensorflow.keras.utils import Sequence
import tensorflow as tf
import os
import json
import random


class DataGenerator(Sequence):
    def __init__(self, error_final, vel_final,
                 batch_size=32,
                 shuffle=True):
        self.error_final = error_final
        self.vel_final = vel_final
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def on_epoch_end(self):
        if self.shuffle:
            temp = [i for i in range(len(self.error_final))]
            random.shuffle(temp)
            #       self.flow_videos = [self.flow_videos[i] for i in temp]
            self.error_final = [self.error_final[i] for i in temp]
            self.vel_final = [self.vel_final[i] for i in temp]

    def __len__(self):
        return int(np.floor(len(self.error_final) / self.batch_size))


    def __getitem__(self, index):
        start_index = index * self.batch_size
#         input_1 = []
        error_reading = []
        vel_reading = []
        i = start_index - 1
        while len(error_reading) < self.batch_size:
            try:
                i += 1
                error = self.error_final[i%len(self.error_final)]
                vel = self.vel_final[i%len(self.vel_final)]
                error_reading.append(error)
                vel_reading.append(vel)

            except Exception as err:
                print(err)
                continue
            
        vel_reading = np.array(vel_reading, dtype = np.float32)
        error_reading = np.array(error_reading, dtype = np.float32)
            
        return error_reading, vel_reading

In [13]:
train_data = DataGenerator(error_final, vel_final, batch_size=32, shuffle=True)

In [14]:
print(train_data.__getitem__(0)[0].shape)

(32, 32, 5)


In [15]:
from tensorflow.keras.callbacks import (
    ModelCheckpoint,
    TensorBoard
)
import tensorflow as tf

In [16]:
callbacks = [tf.keras.callbacks.ModelCheckpoint("/home/uas-dtu/Documents/DL_IBVS/data_script_drone_vel_2/train_{epoch}.tf", verbose = 1,
                                                save_weights_only=True),
             TensorBoard("/home/uas-dtu/Documents/DL_IBVS/data_script_drone_vel_2/combined_logs")]

In [17]:
def mse(y_true, y_pred):
    y_true = tf.convert_to_tensor(y_true, dtype = tf.float32)
    y_pred = tf.convert_to_tensor(y_pred, dtype=tf.float32)
    l1 = tf.reduce_mean(tf.math.pow((y_pred - y_true), 2))
    l2 = tf.reduce_mean(tf.math.abs((y_pred - y_true)))
    loss = l1 + l2
    return loss

In [18]:
from tensorflow.keras.optimizers import Adam
optimizer=Adam(learning_rate=0.0005)

In [19]:
model.compile(optimizer=optimizer,loss=mse, metrics = [tf.keras.metrics.RootMeanSquaredError()])

In [20]:
#model.load_weights("./data_script_drone_vel/train_33.tf")

In [ ]:

model.fit(train_data, epochs=2000, verbose=1, callbacks=callbacks, initial_epoch = 33)

Epoch 34/2000
1271/1271 [==============================] - 1065s 828ms/step - loss: 0.0347 - root_mean_squared_error: 0.0649

Epoch 00034: saving model to /home/uas-dtu/Documents/DL_IBVS/data_script_drone_vel/train_34.tf
Epoch 35/2000
1271/1271 [==============================] - 1043s 820ms/step - loss: 0.0348 - root_mean_squared_error: 0.0649

Epoch 00035: saving model to /home/uas-dtu/Documents/DL_IBVS/data_script_drone_vel/train_35.tf
Epoch 36/2000
1271/1271 [==============================] - 1051s 827ms/step - loss: 0.0343 - root_mean_squared_error: 0.0643

Epoch 00036: saving model to /home/uas-dtu/Documents/DL_IBVS/data_script_drone_vel/train_36.tf
Epoch 37/2000
1271/1271 [==============================] - 1045s 822ms/step - loss: 0.0340 - root_mean_squared_error: 0.0640

Epoch 00037: saving model to /home/uas-dtu/Documents/DL_IBVS/data_script_drone_vel/train_37.tf
Epoch 38/2000
1271/1271 [==============================] - 1051s 827ms/step - loss: 0.0335 - root_mean_squared_error